<a href="https://colab.research.google.com/github/VyomaD/kdm_tech/blob/main/ICP_lab5/source/ICP_lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd
!pip install gensim==3.8.3
import gensim
from gensim import corpora, models, similarities

     |████████████████████████████████| 24.2MB 60.8MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.8MB 67kB/s 
     |████████████████████████████████| 204kB 47.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=d64d03949b6c2723311d4a1891b01ca9d309e29ed4bfc604e7fedcbc8e60c506
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark


In [ ]:
from __future__ import print_function
from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
from pyspark.ml.feature import NGram
from pyspark.ml.feature import Word2Vec

In [ ]:
# creating spark session
spark = SparkSession.builder.appName("TF-IDF").getOrCreate()

In [10]:
sentence = spark.read.option("header", "true").text("TextFiles/*")
sentence.show(5660,truncate=False)
sentenceOutput = sentence

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                       

In [11]:
# creating tokens/words from the sentence data
tokenizer = Tokenizer(inputCol="value", outputCol="words")
wordsData = tokenizer.transform(sentence)

In [12]:
wordsData.show(10,truncate=False)

+----------------------------------------------------------------------+------------------------------------------------------------------------------------+
|value                                                                 |words                                                                               |
+----------------------------------------------------------------------+------------------------------------------------------------------------------------+
|'CARMEN WON'T LAST MORE than a couple of days.' Mason spat out a      |['carmen, won't, last, more, than, a, couple, of, days.', mason, spat, out, a]      |
|chunk of ice and surveyed the poor animal ruefully, then put her      |[chunk, of, ice, and, surveyed, the, poor, animal, ruefully,, then, put, her]       |
|foot in his mouth and proceeded to bite out the ice which clustered   |[foot, in, his, mouth, and, proceeded, to, bite, out, the, ice, which, clustered]   |
|cruelly between the toes.                          

In [13]:
# applying tf on the words data
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# featurizedData.show(truncate=False)

# calculating the IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
# #displaying the results
rescaledData.select("words", "features").show(10,truncate=False)
print(rescaledData)
top_10words_a = rescaledData.select("words", "features").first()


+------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|words                                                                               |features                                                                                                                                                                                                                                         |
+------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|['carmen, wo

In [14]:
spark2 = SparkSession.builder.appName("Lemma Input").getOrCreate()

#Lemmatizer: First we import nltk
word_lemmatizer = WordNetLemmatizer()
#Then we save the data onto the disk
eachWord = wordsData.select("words").collect()
#Where now we can lemmatize each individual word
for word in eachWord: #For each list
  for eachRow in word: #for row
    for i, z in enumerate(eachRow): #for each individual word
      eachRow[i] = word_lemmatizer.lemmatize(z)

#Now we save it into an dataframe
lemmaWord = spark.createDataFrame(eachWord)
lemmaWord.show(truncate=False)

+-------------------------------------------------------------------------------------+
|words                                                                                |
+-------------------------------------------------------------------------------------+
|['carmen, won't, last, more, than, a, couple, of, days.', mason, spat, out, a]       |
|[chunk, of, ice, and, surveyed, the, poor, animal, ruefully,, then, put, her]        |
|[foot, in, his, mouth, and, proceeded, to, bite, out, the, ice, which, clustered]    |
|[cruelly, between, the, toes.]                                                       |
|[, , 'i, never, saw, a, dog, with, a, highfalutin', name, that, ever, wa, worth, a]  |
|[rap,', he, said,, a, he, concluded, his, task, and, shoved, her, aside., 'they]     |
|[just, fade, away, and, die, under, the, responsibility., did, ye, ever, see, one]   |
|[go, wrong, with, a, sensible, name, like, cassiar,, siwash,, or, husky?, no,, sir!] |
|[take, a, look, at, shookum, he

In [15]:
#creating dataframe of input
# applying tf on the words data
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedLemmaData = hashingTF.transform(lemmaWord)
# featurizedLemmaData.show(truncate=False)

# calculating the IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedLemmaData)
rescaledLemmaData = idfModel.transform(featurizedLemmaData)

# #displaying the results
rescaledLemmaData.select("words", "features").show(10,truncate=False)
top_10words_b = rescaledLemmaData.select("words", "features").first()


+------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|words                                                                               |features                                                                                                                                                                                                                       |
+------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|['carmen, won't, last, more, than, a, couple, of, days.', mason, s

In [16]:
spark3 = SparkSession.builder.appName("Ngram").getOrCreate()
#creating dataframe of input

#this section is tokenizing the text, only this time the output column is called ngram
tokenizer = Tokenizer(inputCol="value", outputCol="ngram")
ngramData = tokenizer.transform(sentence)

#Now we can ngram the new dataset, with the output column being words
ngram = NGram(n=2, inputCol="ngram", outputCol="words")
ngramData = ngram.transform(ngramData)
ngramData.show(10, truncate=False)
top_10words_c = ngramData.first()
print(top_10words_c)


+----------------------------------------------------------------------+------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                 |ngram                                                                               |words                                                                                                                                               |
+----------------------------------------------------------------------+------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------+
|'CARMEN WON'T LAST MORE than a couple of days.' Mason spat out a      |['carme

In [17]:
print(top_10words_a.words)
print(top_10words_b.words)
print(top_10words_c.words)

["'carmen", "won't", 'last', 'more', 'than', 'a', 'couple', 'of', "days.'", 'mason', 'spat', 'out', 'a']
["'carmen", "won't", 'last', 'more', 'than', 'a', 'couple', 'of', "days.'", 'mason', 'spat', 'out', 'a']
["'carmen won't", "won't last", 'last more', 'more than', 'than a', 'a couple', 'couple of', "of days.'", "days.' mason", 'mason spat', 'spat out', 'out a']


In [18]:
import gzip
import gensim 
import logging

In [19]:
# process sentences to tokens
processedLines1 = [gensim.utils.simple_preprocess(sentence) for sentence in top_10words_a.words]
#create word list from token using utf8 encoding 
# word_list = [word.encode('utf-8') for words in processedLines for word in words]
word_list1 = [word for words in processedLines1 for word in words]
#check the length of the list
print('Length: ', len(word_list1))
top10_tfidf1 = word_list1[:10]
print(top10_tfidf1)

# process sentences to tokens
processedLines2 = [gensim.utils.simple_preprocess(sentence) for sentence in top_10words_b.words]
#create word list from token using utf8 encoding 
# word_list = [word.encode('utf-8') for words in processedLines for word in words]
word_list2 = [word for words in processedLines2 for word in words]
#check the length of the list
print('Length: ', len(word_list2))
top10_tfidf2 = word_list2[:10]
print(top10_tfidf2)

# process sentences to tokens
processedLines3 = [gensim.utils.simple_preprocess(sentence) for sentence in top_10words_c.words]
#create word list from token using utf8 encoding 
# word_list = [word.encode('utf-8') for words in processedLines for word in words]
word_list3 = [word for words in processedLines3 for word in words]
#check the length of the list
print('Length: ', len(word_list3))
top10_tfidf3 = word_list3[:10]
print(top10_tfidf3)


Length:  11
['carmen', 'won', 'last', 'more', 'than', 'couple', 'of', 'days', 'mason', 'spat']
Length:  11
['carmen', 'won', 'last', 'more', 'than', 'couple', 'of', 'days', 'mason', 'spat']
Length:  21
['carmen', 'won', 'won', 'last', 'last', 'more', 'more', 'than', 'than', 'couple']


In [20]:
model1 = gensim.models.Word2Vec(
    [word_list1], 
    negative=10, 
    iter=50, 
    min_count=1, 
    size=32 #dimension of the word vector
    )


model2 = gensim.models.Word2Vec(
    [word_list2], 
    negative=10, 
    iter=50, 
    min_count=1, 
    size=32 #dimension of the word vector
    )


model3 = gensim.models.Word2Vec(
    [word_list3], 
    negative=10, 
    iter=50, 
    min_count=1, 
    size=32 #dimension of the word vector
    )

In [21]:
print('Finding similar words for top 10 tf-idf words - part 1 ')
for word in top10_tfidf1:
  print(word, model1.wv.most_similar(positive=word))

print('\nFinding similar words for top 10 tf-idf words - part 2 ')
for word in top10_tfidf2:
  print(word, model2.wv.most_similar(positive=word))  

print('\nFinding similar words for top 10 tf-idf words - part 3 ')
for word in top10_tfidf3:
  print(word, model3.wv.most_similar(positive=word))    

Finding similar words for top 10 tf-idf words - part 1 
carmen [('out', 0.28690946102142334), ('couple', 0.27307242155075073), ('days', 0.21944856643676758), ('more', 0.19193454086780548), ('last', 0.12093362212181091), ('than', 0.04368826001882553), ('of', 0.020498380064964294), ('spat', -0.12380953133106232), ('won', -0.13902485370635986), ('mason', -0.2401411086320877)]
won [('mason', 0.2589953541755676), ('of', 0.11770440638065338), ('out', 0.018466148525476456), ('than', -0.021162111312150955), ('couple', -0.022062480449676514), ('last', -0.03151732310652733), ('days', -0.03201949596405029), ('spat', -0.08731769025325775), ('carmen', -0.13902485370635986), ('more', -0.4003359377384186)]
last [('days', 0.24311146140098572), ('than', 0.2377537339925766), ('mason', 0.12687651813030243), ('more', 0.12428043782711029), ('carmen', 0.12093362212181091), ('won', -0.03151732310652733), ('out', -0.1728317141532898), ('couple', -0.21166136860847473), ('of', -0.2580450475215912), ('spat', -0.